In [ ]:
# OBJETIVO DO NOTEBOOK
# 1º Normalizar dados númericos e categóricos
# 2º Caslibrar o número de grupos para o cluster (Otimização)
# 3º Treinar o modelo de cluster 
# 4º Descrever os grupos de modelo de cluster (Centróides)
# 5º Definir a qual grupo uma nova instância pertence

###########################################
# TRABALHO DE NORMALIZAÇÃO
# Data Since: 07/04/2023
# @Autor: Santian, Paulo

!python.exe -m pip install --upgrade pip
!pip install pandas==1.5.2
!pip install matplotlib
!pip install scikit-learn

In [2]:
# Imports
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from scipy.spatial.distance import cdist
from matplotlib import pyplot as plt
from sklearn.cluster import KMeans
from sklearn import preprocessing 
from pickle import dump, load
from google.colab import drive
scaler = StandardScaler()

ABRE O ARQUIVO E REALIZA A LEITURA DO MESMO

In [3]:
# LE O ARQUIVO
drive.mount('/content/drive')
dataframe = pd.read_csv('/content/drive/MyDrive/Faculdade/1º Semestre - 2023/Inteligencia Artificial Avançado/codon_usage.csv', sep=',')
# IMPRIME AS 5 PRIMEIRAS LINHAS
#dataframe.head()

Mounted at /content/drive


<ipython-input-3-7c9f65dce379>:3: DtypeWarning: Columns (5,6) have mixed types. Specify dtype option on import or set low_memory=False.
  dataframe = pd.read_csv('/content/drive/MyDrive/Faculdade/1º Semestre - 2023/Inteligencia Artificial Avançado/codon_usage.csv', sep=',')


In [4]:
# Verifica quais colunas são categóricas
colunas_categoricas = [coluna for coluna in dataframe.columns if dataframe[coluna].dtype == 'object']

#colunas_categoricas

In [5]:
normalizador = preprocessing.MinMaxScaler()

# converte as colunas 'UUU' e 'UUC' para numérico, com valores inválidos convertidos para NaN
dataframe['UUU'] = pd.to_numeric(dataframe['UUU'], errors='coerce')
dataframe['UUC'] = pd.to_numeric(dataframe['UUC'], errors='coerce')

# preenche valores faltantes com a média para colunas numéricas
df_num = dataframe.select_dtypes(include=['float64', 'int64'])
df_num = df_num.fillna(df_num.mean())

# Obter o modelo normalizador e salvar para uso posterior
modelo_normalizador = normalizador.fit(df_num) # O método fit é responsável por obter o modelo que normalizará a base 
dados_num_normalizado = normalizador.fit_transform(df_num) # Responsável por aplicar a normalização

#alternativa
dados_num_normalizado = modelo_normalizador.fit_transform(df_num)
#print(dados_num_normalizado)

#Salvar o modelo normalizador em disco
dump(modelo_normalizador,open('normalizador1.model','wb'))

# preenche valores faltantes com a moda para colunas categóricas
df_cat = dataframe.select_dtypes(include=['object'])
df_cat = df_cat.fillna(df_cat.mode().iloc[0])

In [6]:
# normaliza os dados numéricos
df_num = df_num.select_dtypes(include=['float64','int64'])
#print(df_num)
df_num = pd.DataFrame(scaler.fit_transform(df_num), columns=df_num.columns)

# normaliza os dados categóricos
df_cat = df_cat[['Kingdom']]
df_cat = pd.get_dummies(df_cat)

# concatena as duas partes do dataset
dataframe = pd.concat([df_num, df_cat], axis=1)
# imprime o dataset final
print(dataframe.iloc[:, -1].dtype)

uint8


In [ ]:
distorcoes = []
K = range(1,101)
for k in K: 
  modelo = KMeans(n_clusters=k,random_state=42,n_init='auto').fit(dataframe)
  distorcoes.append(sum(np.min(cdist(dataframe, modelo.cluster_centers_, 'euclidean'), axis=1)) / dataframe.shape[0])
  
plt.plot(distorcoes)

In [8]:
import math
x0 = K[0]
y0 = distorcoes[0]

x1 = K[len(K)- 1]
y1 = distorcoes[len(distorcoes)-1]

distancias = []
for i in range(len(distorcoes)):
  x = K[i]
  y = distorcoes[i]
  numerador = abs((y1-y0)*x - (x1-x0)*y + x1*y0 - y1*x0)
  denominador = math.sqrt((y1-y0)**2 + (x1-x0)**2)
  distancias.append(numerador/denominador)

n_clusters_otimo = K[distancias.index(np.max(distancias))]
print('Numero ideal de clusters:', K[distancias.index(np.max(distancias))])

Numero ideal de clusters: 15


In [9]:
#Treinar o modelo definitivo
from pickle import dump
modelo = KMeans(n_clusters=n_clusters_otimo,random_state=42,n_init='auto').fit(dataframe)
dump(modelo,open('cluster1.modelo','wb'))
type(dataframe.columns)

pandas.core.indexes.base.Index

In [10]:
dados_num_desnormalizados = modelo_normalizador.inverse_transform(dados_num_normalizado)
dados_num_desnormalizados_DF = pd.DataFrame(dados_num_desnormalizados, columns= df_num.columns)
dados_num_desnormalizados_DF.head()

,DNAtype,SpeciesID,Ncodons,UUU,UUC,UUA,UUG,CUU,CUC,CUA,...,CGG,AGA,AGG,GAU,GAC,GAA,GAG,UAA,UAG,UGA
0,0.0,100217.0,1995.0,0.01654,0.01203,0.00050,0.00351,0.01203,0.03208,0.00100,...,0.00451,0.01303,0.03559,0.01003,0.04612,0.01203,0.04361,0.00251,0.00050,0.00000
1,0.0,100220.0,1474.0,0.02714,0.01357,0.00068,0.00678,0.00407,0.02849,0.00204,...,0.00136,0.01696,0.03596,0.01221,0.04545,0.01560,0.04410,0.00271,0.00068,0.00000
2,0.0,100755.0,4862.0,0.01974,0.02180,0.01357,0.01543,0.00782,0.01111,0.01028,...,0.00596,0.01974,0.02489,0.03126,0.02036,0.02242,0.02468,0.00391,0.00000,0.00144
3,0.0,100880.0,1915.0,0.01775,0.02245,0.01619,0.00992,0.01567,0.01358,0.00940,...,0.00366,0.01410,0.01671,0.03760,0.01932,0.03029,0.03446,0.00261,0.00157,0.00000
4,0.0,100887.0,22831.0,0.02816,0.01371,0.00767,0.03679,0.01380,0.00548,0.00473,...,0.00604,0.01494,0.01734,0.04148,0.02483,0.03359,0.03679,0.00000,0.00044,0.00131


In [11]:
# dados_cat_normalizado
df_cat.columns[0].split('_')[1]

'arc'

In [12]:
desnormalizados_cat = df_cat.idxmax(axis=1).to_frame()

desnormalizados_cat['Kingdom'] = desnormalizados_cat[0].apply(lambda x: x.split('_')[1])
desnormalizados_cat = desnormalizados_cat.drop(0,axis=1)

df_desnorm = pd.concat([dados_num_desnormalizados_DF.reset_index(drop=True), desnormalizados_cat.reset_index(drop=True)], axis=1)

df_desnorm

,DNAtype,SpeciesID,Ncodons,UUU,UUC,UUA,UUG,CUU,CUC,CUA,...,AGA,AGG,GAU,GAC,GAA,GAG,UAA,UAG,UGA,Kingdom
0,0.0,100217.0,1995.0,0.01654,0.01203,0.00050,0.00351,0.01203,0.03208,0.00100,...,0.01303,0.03559,0.01003,0.04612,0.01203,0.04361,0.00251,0.00050,0.00000,vrl
1,0.0,100220.0,1474.0,0.02714,0.01357,0.00068,0.00678,0.00407,0.02849,0.00204,...,0.01696,0.03596,0.01221,0.04545,0.01560,0.04410,0.00271,0.00068,0.00000,vrl
2,0.0,100755.0,4862.0,0.01974,0.02180,0.01357,0.01543,0.00782,0.01111,0.01028,...,0.01974,0.02489,0.03126,0.02036,0.02242,0.02468,0.00391,0.00000,0.00144,vrl
3,0.0,100880.0,1915.0,0.01775,0.02245,0.01619,0.00992,0.01567,0.01358,0.00940,...,0.01410,0.01671,0.03760,0.01932,0.03029,0.03446,0.00261,0.00157,0.00000,vrl
4,0.0,100887.0,22831.0,0.02816,0.01371,0.00767,0.03679,0.01380,0.00548,0.00473,...,0.01494,0.01734,0.04148,0.02483,0.03359,0.03679,0.00000,0.00044,0.00131,vrl
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13023,0.0,9601.0,1097.0,0.02552,0.03555,0.00547,0.01367,0.01276,0.02097,0.00820,...,0.01367,0.01094,0.01367,0.02279,0.02005,0.04102,0.00091,0.00091,0.00638,pri
13024,1.0,9601.0,2067.0,0.01258,0.03193,0.01984,0.00629,0.01451,0.05322,0.07644,...,0.00000,0.00048,0.00194,0.01306,0.01838,0.00677,0.00242,0.00097,0.01887,pri
13025,1.0,9602.0,1686.0,0.01423,0.03321,0.01661,0.00356,0.01127,0.05042,0.09609,...,0.00000,0.00000,0.00178,0.01661,0.02788,0.00297,0.00356,0.00119,0.02017,pri
13026,0.0,9606.0,40662582.0,0.01757,0.02028,0.00767,0.01293,0.01319,0.01959,0.00715,...,0.01217,0.01196,0.02178,0.02510,0.02896,0.03959,0.00099,0.00079,0.00156,pri


In [153]:
#Inferência
# Le os novos dados
newDataframe = pd.read_csv('newData.csv', sep=',')
newDataframe = newDataframe.drop(['SpeciesName'], axis=1)
newDataframe

,Kingdom,DNAtype,SpeciesID,Ncodons,UUU,UUC,UUA,UUG,CUU,CUC,...,CGG,AGA,AGG,GAU,GAC,GAA,GAG,UAA,UAG,UGA
0,mam,1,9829,12160,0.02656,0.04268,0.00789,0.00025,0.01176,0.03775,...,0,0.00263,0,0.00403,0.02492,0.01546,0.00033,0,0,0.03002


In [140]:
#Normaliza os dados
# Verifica quais colunas são categóricas
newColunas_categoricas = [coluna for coluna in newDataframe.columns if newDataframe[coluna].dtype == 'object']
newColunas_categoricas

['Kingdom']

In [141]:
#le o normalizador salvo anteriormente
newNormalizador = load(open('/content/normalizador1.model', 'rb'))

In [ ]:
# preenche valores faltantes com a média para colunas numéricas
newDf_num = newDataframe.select_dtypes(include=['float64', 'int64'])
newDf_num = newDf_num.fillna(newDf_num.mean())


# normaliza os dados categóricos
newDf_cat = newDataframe[['Kingdom']]
newDf_cat = newDf_cat.fillna(newDf_cat.mode().iloc[0])
newDf_cat_normalizado = pd.get_dummies(newDf_cat)
newDf_num

In [ ]:
#Normaliza os dados num
#newDf_num_normalizado = normalizador.transform(newDf_num)
newDf_num_normalizado = newNormalizador.transform(newDf_num)
newDf_num_normalizado = pd.DataFrame(newDf_num_normalizado, columns=newDf_num.columns) 
newDf_num_normalizado

In [ ]:
#Junta os dois dados cat e num
# newDF_normalizado = pd.concat([newDf_num_normalizado, newDf_cat_normalizado], axis=1)
newDf_cat_normalizado.iloc[:, -1].name

In [145]:
# newDF_normalizado_com_reinos = pd.DataFrame({})

# Cria um dataframe com os reinos
df_reinos = pd.DataFrame({
    'Kingdom_arc':   [],
    'Kingdom_bct':   [],
    'Kingdom_inv':   [],
    'Kingdom_mam':   [],
    'Kingdom_phg':   [],
    'Kingdom_plm':   [],
    'Kingdom_pln':   [],
    'Kingdom_pri':   [],
    'Kingdom_rod':   [],
    'Kingdom_vrl':   [],
    'Kingdom_vrt':   []
})


#JUNTA OS DADOS NUMERICOS COM AS COLUNAS CATEGORICAS GERADAS

newDF_normalizado_com_reinos = pd.concat([newDf_num_normalizado, df_reinos], axis=1)
# newDF_normalizado_com_reinos
newDF_normalizado = newDF_normalizado_com_reinos.fillna(0)
newDF_normalizado = pd.concat([newDF_normalizado, newDf_cat_normalizado], axis=1 ) 

#Salva a ordem das colunas em uma variavel lista
lst_nameColumns = list(newDF_normalizado_com_reinos.columns)
#print(lst_nameColumns)

#Agrupa as colunas duplicadas
newDF_normalizado = newDF_normalizado.groupby(newDF_normalizado.columns, axis=1).last()

#Reorganiza as colunas para o padrão
newDF_normalizado = newDF_normalizado.reindex(columns=lst_nameColumns)
# print(newDF_normalizado)

In [ ]:
#Desnormaliza os dados
newDF_normalizado_final = newDF_normalizado.drop(columns=list(df_reinos))
newDF_desnormalizado =  newNormalizador.inverse_transform(newDF_normalizado_final)

#Transforma em Data Frame
newDF_desnormalizado = pd.DataFrame(newDF_desnormalizado, columns= newDf_num_normalizado.columns)

# Adiciona a coluna categorica
newDF_desnormalizado['Kingdom'] = newDf_cat['Kingdom']
print(newDF_desnormalizado)


In [ ]:
#Carrega o modelo de cluster
modelo_cluster = load(open('/content/cluster1.modelo','rb'))

# Previsão dos clusters dos novos dados normalizados
novos_dados_cluster = modelo_cluster.predict(newDF_normalizado)

# Adiciona a coluna de clusters ao dataframe dos novos dados desnormalizados
newDF_desnormalizado['cluster'] = novos_dados_cluster

print(newDF_desnormalizado)

In [ ]:
newDF_desnormalizado